In [1]:
import sys
import os
import dolfin
sys.path.append(os.getcwd()[:-6])

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

In [3]:
folder = "test-folder-prescribed-u"
path = os.path.join("../raw-data", folder)

In [4]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.generate_slab_mesh(
    dims=(200, 6, 200), elems=(40, 3, 30), save=True, periodic=True
)
model.sim_p(save=True)
model.sim_bst(final_time=1, num_steps=5, save=True)

Solving linear variational problem.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 7.181e+05 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 4.009e-09 (tol = 1.000e-10) r (rel) = 5.583e-15 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Finished time step 1/5 (20%)

No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.606e+05 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 3.125e-09 (tol = 1.000e-10) r (rel) = 1.946e-14 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Finished time step 2/5 (40%)

No Jacobian form specified for nonlinear varia

In [5]:
model.boundaries
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries

In [9]:
from ipygany import Scene, PolyMesh

mesh = PolyMesh.from_vtk(os.path.join(model.folder_path, 'p/p000000.vtu'))
mesh.default_color = 'gray'

scene = Scene([mesh])
scene